In [8]:
%matplotlib widget
import numpy as np
from matplotlib import pyplot as plt
from ipywidgets import interact
from scipy import stats
import copy

from cvrp_simulation.scenario_generator import SampleDynamicBenchmark
from cvrp_simulation.simulator import CVRPSimulation


In [20]:
def get_cmap(n, name='nipy_spectral'):
    '''Returns a function that maps each index in 0, 1, ..., n-1 to a distinct 
    RGB color; the keyword argument name must be a standard mpl colormap name.'''
    return plt.cm.get_cmap(name, n)

In [21]:
def plot_vehicle_route(all_states):
    plt.scatter(all_states[0].current_state.depot_position[0], all_states[0].current_state.depot_position[1], marker='s', c='r')
    plt.text(all_states[0].current_state.depot_position[0], all_states[0].current_state.depot_position[1], 'depot')
    x = [s.current_state.current_vehicle_position[0] for s in all_states]
    y = [s.current_state.current_vehicle_position[1] for s in all_states]
    plt.plot(x, y, marker='.')
    for i in range(len(x)):
        plt.text(x[i], y[i], str(i))
    plt.grid()

In [25]:
depot_position_rv = stats.uniform(loc=0, scale=1)
vehicle_position_rv = stats.uniform(loc=0, scale=1)
customer_positions_rv = stats.uniform(loc=0, scale=1)
customer_demands_rv = stats.randint(low=0, high=10)
customer_times_rv = stats.randint(low=0, high=100)
vehicle_velocity = 10
vrp_size = 20
initial_vehicle_capacity = vrp_size*2
benchmark_generator = SampleDynamicBenchmark(depot_position_rv=depot_position_rv, vehicle_position_rv=vehicle_position_rv, 
                                            vehicle_capacity=initial_vehicle_capacity, vehicle_velocity=vehicle_velocity,
                                            customer_positions_rv=customer_positions_rv, customer_demands_rv=customer_demands_rv,
                                            customer_times_rv=customer_times_rv,
                                            vrp_size=vrp_size)
seed = 5
sim = CVRPSimulation(max_customers=vrp_size, problem_generator=benchmark_generator, allow_noop = True)
sim.seed(seed)
color_map = get_cmap(10)
depot_color = color_map(0)
vehicle_color = color_map(1)

C:\Users\ros1tv\Anaconda3\envs\waferfab_test\lib\site-packages\gym-0.15.7-py3.7.egg\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [26]:
obs = sim.reset()
# in this case each time we choose action 0 since the available actions change each time
# the number of available customers changes
tot_reward = 0
done = False
all_states = [copy.deepcopy(sim)]
all_observations = [copy.deepcopy(obs)]
timeline = [0]
while not done:
    available_actions = obs['action_mask']
    action_chosen = np.random.choice(np.flatnonzero(available_actions), 1)[0]
    obs, reward, done, _ = sim.step(action_chosen)
    tot_reward += reward
    all_states.append(copy.deepcopy(sim))
    all_observations.append(copy.deepcopy(obs))
    timeline.append(int(np.ceil(sim.current_time)))
print(f"finished random run, total reward {tot_reward}")
plt.close()
plot_vehicle_route(all_states)

finished random run, total reward -12.683129104860264


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [27]:
@interact
# for range (min, max, step) , this gives a slider
# for drop down give a list of the options
def plot_current_state(t=(0, len(timeline)-1, 1)):
    plt.close()
    cur_sim = all_states[t]
    observation_space = all_observations[t]
    num_customers = cur_sim.current_state.customer_positions.shape[0]
    plt.scatter([], [], label='opened, demand<capacity',c='b')
    plt.scatter([], [], label='visited', c='g')
    plt.scatter([], [], label='opened, demand>capacity', c='m')
    plt.scatter([], [], label='closed', c='r')
    plt.scatter(cur_sim.current_state.depot_position[0], cur_sim.current_state.depot_position[1], marker='s', color=depot_color)
    
    for i_c in range(num_customers):
        customer_position = cur_sim.current_state.customer_positions[i_c, :]
        # opened customer
        plt.text(customer_position[0], customer_position[1],str(i_c))
        if cur_sim.current_state.customer_visited[i_c]:
            plt.scatter(customer_position[0], customer_position[1], marker='o', c='g')
        elif cur_sim.current_state.customer_times[i_c]<= timeline[t]:
            if cur_sim.current_state.customer_demands[i_c]<=cur_sim.current_state.current_vehicle_capacity:
                plt.scatter(customer_position[0], customer_position[1], marker='o', c='b')
            else:
                plt.scatter(customer_position[0], customer_position[1], marker='o', c='m')
                print(f"customer demand:{cur_sim.current_state.customer_demands[i_c]} , capacity:{cur_sim.current_state.current_vehicle_capacity}")
        elif cur_sim.current_state.customer_times[i_c]> timeline[t]:
            plt.scatter(customer_position[0], customer_position[1], marker='o', c='r')
        else:
            plt.scatter(customer_position[0], customer_position[1], marker='o', c='k')
    n_e = 0
    for i_c_o, obs_customer_position in enumerate(observation_space['customer_positions']):
        if obs_customer_position.all()!=0:
            plt.scatter(obs_customer_position[0], obs_customer_position[1], marker='.', c='w')
            n_e += 1
    # plot vehicle position 
    plt.scatter(cur_sim.current_state.current_vehicle_position[0], cur_sim.current_state.current_vehicle_position[1], marker='*', color=vehicle_color, s=60)
    plt.grid()
    plt.legend(loc='best')
    plt.title(f"t={timeline[t]} - customer locations")
    plt.show()
    print(f"number of available customers in observation:{n_e}")

interactive(children=(IntSlider(value=13, description='t', max=27), Output()), _dom_classes=('widget-interact'…

In [19]:
@interact
# for range (min, max, step) , this gives a slider
# for drop down give a list of the options
def plot_current_obs(t=list(range(10))):
    plt.close()
    plt.scatter([2], [3+t])
    plt.xlim([0, 10])
    plt.show()

interactive(children=(Dropdown(description='t', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9), value=0), Output()), _…